# Sprot 数据预处理
## 1. 导入必要的包

In [1]:
import numpy as np
import pandas as pd
import random
import time
import gzip
import re
from Bio import SeqIO
import datetime

## 2. 定义必要的函数

In [2]:
#从gizp读取含有EC号的数据
def read_file_from_gzip(file_in_path, file_out_path):
    counter = 0
    saver = 0 
    reslist = []
#     reslist.append(['id', 'description', 'seq'])
    with gzip.open(file_in_path, "rt") as handle:
        counter+=1
        for record in SeqIO.parse(handle, 'swiss'):
            res = process_record(record)

            if counter %10000==0:
                print('lines:{0}/{1}'.format(saver, counter))

            if len(res) >0:
                reslist.append(res)
                saver +=1
            else:
                continue

    result = np.array(reslist,dtype=object)
    np.savetxt(file_out_path, result, delimiter="\t", fmt='%s')
 
# 提取单条含有EC号的数据
def process_record(record):
    description = record.description
    if 'EC=' in description:
        id = record.id
        name = record.name
        seq = record.seq
        ec = str(re.findall(r"EC=[0-9,.\-;]*",description)).replace('EC=','').replace('\'','').replace(']','').replace('[','').replace(';','')
        date_integraged = record.annotations.get('date')
        date_sequence_update = record.annotations.get('date_last_sequence_update')
        date_annotation_update = record.annotations.get('date_last_annotation_update')
        return [id, ec, date_integraged, date_sequence_update, date_annotation_update,  seq]
    else:
        return []
    
#将表格存储为fasta文件    
def table_2_fasta(table, file_out):
    file = open(file_out, 'w')
    for index, row in table.iterrows():
        file.write('>{0}\n'.format(row['id']))
        file.write('{0}\n'.format(row['seq']))
    file.close()
    print('Write finished')

# 将给定的数据随机划分为2份
def split_random(data):
    index_ref = random.sample(range(0,len(data)), int(len(data)/2))  #创建随机index
    ref = data.iloc[index_ref]     #筛选ref
    query = data.iloc[~data.index.isin(index_ref)] # 剩下的是Query
    table_2_fasta(ref, './data/sprot_with_ec_ref.fasta')   #写入文件ref fasta
    table_2_fasta(query, './data/sprot_with_ec_query.fasta') #写入文件query fasta
    return query, ref

# 按时序前一半后一半划分
def split_time_half(data):
    index_ref = range(int(len(data)/2))
    ref = data.iloc[index_ref]     #筛选ref
    query = data.iloc[~data.index.isin(index_ref)] # 剩下的是Query
    table_2_fasta(ref, './data/sprot_with_ec_ref.fasta')   #写入文件ref fasta
    table_2_fasta(query, './data/sprot_with_ec_query.fasta') #写入文件query fasta

## 3. 提取含有EC号的数据，并保存到文件

In [4]:
start =  time.process_time()
in_filepath = r'./data/uniprot_sprot.dat.gz'
out_filepath = r'./data/sprot_with_ec.tsv'
read_file_from_gzip(file_in_path=in_filepath, file_out_path=out_filepath)
end =  time.process_time()
print('finished use time %6.3f s' % (end - start))

finished use time 206.772 s


## 4. 加载处理完的数据

In [5]:
sprot = pd.read_csv('./data/sprot_with_ec.tsv', sep='\t',names=['id', 'ec_number', 'date_integraged','date_sequence_update','date_annotation_update','seq']) #读入文件
sprot.date_integraged = pd.to_datetime(sprot['date_integraged'])
sprot.date_sequence_update = pd.to_datetime(sprot['date_sequence_update'])
sprot.date_annotation_update = pd.to_datetime(sprot['date_annotation_update'])
sprot.head(3)

,id,ec_number,date_integraged,date_sequence_update,date_annotation_update,seq
0,Q6GZW6,3.6.4.-,2011-06-28,2004-07-19,2020-08-12,MDTSPYDFLKLYPWLSRGEADKGTLLDAFPGETFEQSLASDVAMRR...
1,Q6GZV6,2.7.11.1,2011-06-28,2004-07-19,2021-02-10,MATNYCDEFERNPTRNPRTGRTIKRGGPVFRALERECSDGAARVFP...
2,Q197B6,2.7.11.-,2009-06-16,2006-07-11,2020-08-12,MPLSVFAEEFAEKSVKRYIGQGLWLPCNLSDYYYYQEFHDEGGYGS...


#### 4.1 按时间节点处理
##### 4.1.1 按2018年2月28日为节点划分

In [4]:
#确定阈值
thres = datetime.datetime(2018, 2, 28, 0, 0)

#划分数据
query = sprot[sprot.date_integraged > thres ].sort_values(by='date_integraged')
ref = sprot[sprot.date_integraged <= thres ].sort_values(by='date_integraged')

#保存数据
table_2_fasta(query, './results/sprot_query_201802.fasta')
table_2_fasta(ref, './results/sprot_ref_201802.fasta')

#建库并比对
! ./diamond makedb --in ./results/sprot_ref_201802.fasta -d ./results/sprot_ref_201802.dmnd     #建库
! ./diamond blastp -d ./results/sprot_ref_201802.dmnd  -q ./results/sprot_query_201802.fasta -o ./results/sprot_query_201802_match.tsv -b5 -c1 -k 1   #生成比对文件

#读入比对结果
qure_res_data = pd.read_csv('./results/sprot_query_201802_match.tsv', sep='\t', names=['id', 'sseqid', 'pident', 'length','mismatch','gapopen','qstart','qend','sstart','send','evalue','bitscore'])
print('query results data numbers:{0}'.format(len(qure_res_data)))

Write finished
Write finished
diamond v2.0.8.146 (C) Max Planck Society for the Advancement of Science
Documentation, support and updates available at http://www.diamondsearch.org

#CPU threads: 32
Scoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)
Database input file: ./results/sprot_ref_201802.fasta
Opening the database file...  [0.035s]
Loading sequences...  [0.357s]
Masking sequences...  [0.263s]
Writing sequences...  [0.072s]
Hashing sequences...  [0.028s]
Loading sequences...  [0s]
Writing trailer...  [0.001s]
Closing the input file...  [0.003s]
Closing the database file...  [0.929s]
Database hash = 569a323cc478a3a399869f1002c5fefb
Processed 266465 sequences, 108446438 letters.
Total time = 1.693s
diamond v2.0.8.146 (C) Max Planck Society for the Advancement of Science
Documentation, support and updates available at http://www.diamondsearch.org

#CPU threads: 32
Scoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)
Temporary directory: .

##### 4.1.2 拼合原始数据和比对数据

In [5]:
# 拼合原始数据和比对数据
sprot_extended = sprot
sprot_extended['seqlength'] = sprot_extended['seq'].map(lambda x : len(x))
sprot_extended = pd.merge(sprot_extended,qure_res_data, on=['id'], how='inner')
sprot_extended['coverage'] = sprot_extended.apply(lambda x: x['length']/x['seqlength'], axis=1)

sprot_extended.head(3)

,id,ec_number,date_integraged,date_sequence_update,date_annotation_update,seq,seqlength,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,coverage
0,M5AWY0,1.14.14.-,2018-06-20,2013-05-29,2020-02-26,MQAGFFHTPYNLPTRTARQMFDWSLKLAQVCDEAGFADFMIGEHST...,363,Q6STM1,92.8,361,26,0,1,361,1,361,5.020000e-268,728.0,0.99449
1,A5CAL1,"1.1.1.26, 1.1.1.28, 1.1.1.79, 1.1.1.-",2020-06-17,2007-06-12,2021-04-07,MESIGVLLTCPMNPYLEQELDKRFKLFRFWDFPSANDLFREHSNSI...,313,Q65CJ7,78.0,313,69,0,1,313,1,313,1.330000e-176,493.0,1.00000
2,A0A0N9HQ36,1.14.20.8,2021-02-10,2016-01-20,2021-02-10,MGSTAPLRLPVIDLSMKNLKPGTTSWNSVRTQVREALEEYGCFEAV...,310,Q945B6,41.6,310,170,5,8,307,13,321,4.660000e-78,243.0,1.00000


##### 4.1.3 添加比对结果的EC号

In [6]:
# 添加比对结果的EC号

counter =0
resArray =[]

for i in range(len(sprot_extended)):
    counter+=1
    mn = sprot[sprot['id']== sprot_extended['sseqid'][i]]['ec_number'].values
    resArray.append(mn)
sprot_extended['sresults_ec']=np.array(resArray)  

sprot_extended['iscorrect'] = sprot_extended[['ec_number', 'sresults_ec']].apply(lambda x: x['ec_number'] == x['sresults_ec'], axis=1) #判断EC号是否一致

##### 4.1.4 计算指标 

In [28]:
correct = sum(sprot_extended['iscorrect'])
find  = len(sprot_extended)
total = len(query)
print('Total query records are: {0}'.format(total))
print('Matched records are: {0}'.format(find))
print('Recall: {0}({1}/{2})'.format(correct/total, correct, total))
print('Pricision: {0}({1}/{2})'.format(correct/find, correct, find))
print('Recall: {0}({1}/{2})'.format(find/total, find, total))

Total query records are: 3771
Matched records are: 3246
Recall: 0.5242641209228321(1977/3771)
Pricision: 0.6090573012939002(1977/3246)
Recall: 0.8607796340493238(3246/3771)


##### 4.1.4 使用DeepEC进行计算

In [17]:
! python ./deepec/deepec.py -i ./results/sprot_query_201802.fasta -o ./results/deepec/

2021-04-19 01:33:06.503896: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-04-19 01:36:06.861786: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-19 01:36:06.876688: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-04-19 01:36:06.918909: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
2021-04-19 01:36:06.918946: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: gpu1.tibd.net
2021-04-19 01:36:06.918959: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: gpu1.tibd.net
2021-04-19 01:36:06.931873: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version 

##### 4.1.5 加载DeepEC的预测结果

In [21]:
deepec_results = pd.read_csv('./results/sprot_query_201802/DeepEC_Result.txt', sep='\t',names=['id', 'ec_number'], header=0) #读入文件
deepec_results.ec_number=deepec_results.apply(lambda x: x['ec_number'].replace('EC:',''), axis=1)
deepec_results.head(3)

,id,ec_number
0,Q46085,2.7.7.7
1,Q3K9A2,1.5.1.38
2,Q65YX0,1.5.1.38


##### 4.1.6 添加比对结果的EC号

In [22]:

counter =0
resArray =[]

for i in range(len(deepec_results)):
    counter+=1
    mn = sprot[sprot['id']== deepec_results['id'][i]]['ec_number'].values
    resArray.append(mn)
deepec_results['sresults_ec']=np.array(resArray)  

deepec_results['iscorrect'] = deepec_results[['ec_number', 'sresults_ec']].apply(lambda x: x['ec_number'] == x['sresults_ec'], axis=1) #判断EC号是否一致

deepec_results.head(3)

,id,ec_number,sresults_ec,iscorrect
0,Q46085,2.7.7.7,3.4.24.3,False
1,Q3K9A2,1.5.1.38,1.5.1.42,False
2,Q65YX0,1.5.1.38,1.5.1.42,False


##### 4.1.7 计算DeepEC的指标

In [27]:
correct = sum(deepec_results['iscorrect'])
find  = len(deepec_results)
total = len(query)
print('Total query records are: {0}'.format(total))
print('Matched records are: {0}'.format(find))
print('Accuracy: {0}({1}/{2})'.format(correct/total, correct, total))
print('Pricision: {0}({1}/{2})'.format(correct/find, correct, find))
print('Recall: {0}({1}/{2})'.format(find/total, find, total))

Total query records are: 3771
Matched records are: 1022
Accuracy: 0.10580747812251393(399/3771)
Pricision: 0.3904109589041096(399/1022)
Recall: 0.27101564571731634(1022/3771)


### 4.2. 随机进行处理

#### 4.2.1 按随机方法划分数据，1/2 REF， 1/2 QUERY

In [35]:
query, ref = split_random(sprot)
! ./diamond makedb --in ./data/sprot_with_ec_ref.fasta -d ./data/sprot_with_ec_ref.dmnd     #建库
! ./diamond blastp -d ./data/sprot_with_ec_ref.dmnd  -q ./data/sprot_with_ec_query.fasta -o ./data/sprot_with_ec_query_result.tsv -b5 -c1 -k 1   #生成比对文件

Write finished
Write finished
diamond v2.0.8.146 (C) Max Planck Society for the Advancement of Science
Documentation, support and updates available at http://www.diamondsearch.org

#CPU threads: 32
Scoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)
Database input file: ./data/sprot_with_ec_ref.fasta
Opening the database file...  [0.019s]
Loading sequences...  [0.189s]
Masking sequences...  [0.139s]
Writing sequences...  [0.026s]
Hashing sequences...  [0.014s]
Loading sequences...  [0s]
Writing trailer...  [0.001s]
Closing the input file...  [0.002s]
Closing the database file...  [0.51s]
Database hash = 34d2f56d56e519bfefff960a5bbd0ee4
Processed 135118 sequences, 55338380 letters.
Total time = 0.904s
diamond v2.0.8.146 (C) Max Planck Society for the Advancement of Science
Documentation, support and updates available at http://www.diamondsearch.org

#CPU threads: 32
Scoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)
Temporary directory: ./dat

#### 4.2.2 拼合原始数据和比对数据

In [39]:
qure_res_data = pd.read_csv('./data/sprot_with_ec_query_result.tsv', sep='\t', names=['id', 'sseqid', 'pident', 'length','mismatch','gapopen','qstart','qend','sstart','send','evalue','bitscore'])
print('query results data numbers:{0}'.format(len(qure_res_data)))
query['seqlength'] = query['seq'].map(lambda x : len(x))
qure_res_data = pd.merge(query,qure_res_data, on=['id'], how='inner')
qure_res_data['coverage'] = qure_res_data.apply(lambda x: x['length']/x['seqlength'], axis=1)
qure_res_data.head(3)

query results data numbers:133052


<ipython-input-39-fa89c01faa3c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query['seqlength'] = query['seq'].map(lambda x : len(x))


,id,ec_number,date_integraged,date_sequence_update,date_annotation_update,seq,seqlength,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,coverage
0,Q196W5,3.4.24.-,2009-06-16,2006-07-11,2021-04-07,MSVDSFTSRLAVVMTAVVLVWWAQALPVPSPRRGESDCDAACRKFL...,363,O18767,30.3,323,166,8,8,323,5,275,1.700000e-34,134.0,0.889807
1,Q8GBW6,2.1.3.1,2005-02-01,2007-01-23,2021-04-07,MAENNNLKLASTMEGRVEQLAEQRQVIEAGGGERRVEKQHSQGKQT...,611,P53003,53.3,510,220,4,31,524,39,546,2.070000e-175,510.0,0.834697
2,O55761,3.4.24.-,2009-06-16,1998-06-01,2021-04-07,MKINTTDVRTAIYHSLQQWQSVSLLTFKEEYYDENADIKVSFVKGK...,264,P09237,39.7,136,73,5,15,142,132,266,5.460000e-22,93.6,0.515152


#### 4.2.3 添加比对结果的EC号

In [40]:
counter =0
resArray =[]

for i in range(len(qure_res_data)):
    counter+=1
    mn = sprot[sprot['id']== qure_res_data['sseqid'][i]]['ec_number'].values
    resArray.append(mn)
qure_res_data['sresults_ec']=np.array(resArray) 
qure_res_data.head(3)

,id,ec_number,date_integraged,date_sequence_update,date_annotation_update,seq,seqlength,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,coverage,sresults_ec
0,Q196W5,3.4.24.-,2009-06-16,2006-07-11,2021-04-07,MSVDSFTSRLAVVMTAVVLVWWAQALPVPSPRRGESDCDAACRKFL...,363,O18767,30.3,323,166,8,8,323,5,275,1.700000e-34,134.0,0.889807,3.4.24.-
1,Q8GBW6,2.1.3.1,2005-02-01,2007-01-23,2021-04-07,MAENNNLKLASTMEGRVEQLAEQRQVIEAGGGERRVEKQHSQGKQT...,611,P53003,53.3,510,220,4,31,524,39,546,2.070000e-175,510.0,0.834697,6.4.1.3
2,O55761,3.4.24.-,2009-06-16,1998-06-01,2021-04-07,MKINTTDVRTAIYHSLQQWQSVSLLTFKEEYYDENADIKVSFVKGK...,264,P09237,39.7,136,73,5,15,142,132,266,5.460000e-22,93.6,0.515152,3.4.24.23


#### 4.2.4. 计算指标

In [43]:
qure_res_data['iscorrect'] = qure_res_data[['ec_number', 'sresults_ec']].apply(lambda x: x['ec_number'] == x['sresults_ec'], axis=1) #判断EC号是否一致

correct = sum(qure_res_data['iscorrect'])
find  = len(qure_res_data)
total = len(query)
print('Total query records are: {0}'.format(total))
print('Matched records are: {0}'.format(find))
print('Accuracy: {0}({1}/{2})'.format(correct/total, correct, total))
print('Pricision: {0}({1}/{2})'.format(correct/find, correct, find))
print('Recall: {0}({1}/{2})'.format(find/total, find, total))

Total query records are: 135118
Matched records are: 133052
Recall: 0.9507911603191285(128469/135118)
Pricision: 0.9655548206716171(128469/133052)
Recall: 0.9847096611850382(133052/135118)


#### 4.1.4 使用DeepEC进行计算

In [1]:
! python ./deepec/deepec.py -i ./data/sprot_with_ec_query.fasta -o ./results/deepec1/

2021-04-19 03:16:51.900139: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
^C


#### 4.2.5 加载DeepEC的预测结果

In [3]:
deepec_results = pd.read_csv('./results/deepec1/DeepEC_Result.txt', sep='\t',names=['id', 'ec_number'], header=0) #读入文件
deepec_results.ec_number=deepec_results.apply(lambda x: x['ec_number'].replace('EC:',''), axis=1)
deepec_results.head(3)

,id,ec_number
0,Q6GZV6,2.7.11.1
1,Q8GBW6,2.1.3.1
2,E7CY70,3.2.1.55


#### 4.2.6 添加比对结果的EC号

In [6]:
counter =0
resArray =[]

for i in range(len(deepec_results)):
    counter+=1
    mn = sprot[sprot['id']== deepec_results['id'][i]]['ec_number'].values
    resArray.append(mn)
deepec_results['sresults_ec']=np.array(resArray)  

deepec_results['iscorrect'] = deepec_results[['ec_number', 'sresults_ec']].apply(lambda x: x['ec_number'] == x['sresults_ec'], axis=1) #判断EC号是否一致

deepec_results.head(3)

,id,ec_number,sresults_ec,iscorrect
0,Q6GZV6,2.7.11.1,2.7.11.1,True
1,Q8GBW6,2.1.3.1,2.1.3.1,True
2,E7CY70,3.2.1.55,3.2.1.-,False


####   4.2.7 计算DeepEC的指标

In [9]:
correct = sum(deepec_results['iscorrect'])
find  = len(deepec_results)
total = 135118
print('Total query records are: {0}'.format(total))
print('Matched records are: {0}'.format(find))
print('Accuracy: {0}({1}/{2})'.format(correct/total, correct, total))
print('Pricision: {0}({1}/{2})'.format(correct/find, correct, find))
print('Recall: {0}({1}/{2})'.format(find/total, find, total))

Total query records are: 135118
Matched records are: 114988
Accuracy: 0.6778519516274664(91590/135118)
Pricision: 0.7965178975197412(91590/114988)
Recall: 0.8510191092230495(114988/135118)
